In [25]:
import tensorflow as tf
import string
import requests
import pandas as pd

In [40]:
response = ""
with open('bieber.txt', 'r') as file:
    response = file.read()
response

'What do you mean?\nOh, oh, oh\nWhen you sometimes say yes\nBut you sometimes say no\nWhat do you mean?\nHey, yeah\nWhen you don\'t want me to move\nBut you tell me to go\nWhat do you mean?\nOh\nWhat do you mean?\nSaid you\'re running out of time\nWhat do you mean?\nOh, oh, oh\nWhat do you mean?\nBetter make up your mind\nWhat do you mean? You\'re so indecisive, what I\'m saying\nTrying to catch the beat, make up your heart\nDon\'t know if you\'re happy or complaining\nDon\'t want for us to end\nWhere do I start?\nFirst you wanna go to the left and you want to turn right (ooh)\nWanna argue all day, hangin\' out all night (oh)\nFirst you\'re up, then you\'re down and in between\nOh, I really want to know What do you mean?\nOh, oh, oh\nWhen you sometimes say yes\nBut you sometimes say no\nWhat do you mean?\nHey, yeah\nWhen you don\'t want me to move\nBut you tell me to go\nWhat do you mean?\nOh, what do you mean?\nSaid you\'re running out of time\nWhat do you mean?\nOh, oh, oh\nWhat do y

In [41]:
data=response.splitlines()
print(data[:10]) # 10 sample

['What do you mean?', 'Oh, oh, oh', 'When you sometimes say yes', 'But you sometimes say no', 'What do you mean?', 'Hey, yeah', "When you don't want me to move", 'But you tell me to go', 'What do you mean?', 'Oh']


In [42]:
len(data)

3715

In [43]:
# Total word number is:
len(" ".join(data))

143695

In [44]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences


#### LSTM model and train test preparation

In [45]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [47]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text[:10]
# These number actually comes from vocabulary
# it assigns number to each words

[[30, 26, 1, 90],
 [6, 6, 6],
 [45, 1, 164, 55, 354],
 [32, 1, 164, 55, 20],
 [30, 26, 1, 90],
 [83, 18],
 [45, 1, 14, 47, 4, 5, 267],
 [32, 1, 68, 4, 5, 48],
 [30, 26, 1, 90],
 [6]]

In [48]:
wc=tokenizer.word_counts
# word frequency

In [49]:
wi=tokenizer.word_index

In [50]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1 # always adding plus one for tensorflow

Number of unique words and total vocab size: 1979


In [55]:
x=["play this song"]

In [56]:
tokenizer.texts_to_sequences(x)

[[517, 58, 462]]

### Prepare data for training

In [57]:
 encoded_text[:10] # 10 sample

[[30, 26, 1, 90],
 [6, 6, 6],
 [45, 1, 164, 55, 354],
 [32, 1, 164, 55, 20],
 [30, 26, 1, 90],
 [83, 18],
 [45, 1, 14, 47, 4, 5, 267],
 [32, 1, 68, 4, 5, 48],
 [30, 26, 1, 90],
 [6]]

In [58]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])
#             print(i[:j]) # if you want to check data

#### Paddding

In [59]:
max_length=20
#max length of line is 20 token per line in our poetry

In [60]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to 20
sequences[:5] # 5 sample

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  30,  26],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  30,  26,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   6,   6],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  45,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  45,   1, 164]], dtype=int32)

In [62]:
sequences.shape

(22129, 20)

In [63]:
X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X[:5]) # 5 sample
print("-"*30)
print("X values")
print(y[:5]) # 5 sample

X values
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30 26]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 45]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 45  1]]
------------------------------
X values
[ 26   1   6   1 164]


In [64]:
X.shape,y.shape

((22129, 19), (22129,))

In [65]:
y=to_categorical(y,num_classes=vocab_size)
#since unique word number is vocab_size, thus there is vocab_size classes
print(y[:5]) # 5 sample
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (22129, 1979)


In [66]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (22129, 19)


19

#### Build Model
- We will build a simple LSTM model

In [67]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#### You can use dropout() , You will see that Training will be slightly slower trend in convergence, maybe lower final accuracy

In [69]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(Dropout(0.2))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

####  Dropout can be applied to the input and recurrent connections of the memory units with the LSTM precisely and separately.
#### LSTM-specific dropout may have more pronounced effect on the convergence of the network than the layer-wise dropout.

In [70]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

#### LSTM and CNN can be used together

In [71]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(Conv1D(filters=32,kernel_size=3, padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

In [72]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [73]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [74]:
model.fit(X,y,batch_size=32,epochs=150)

Epoch 1/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - accuracy: 0.0488 - loss: 6.0249
Epoch 2/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.0582 - loss: 5.4327
Epoch 3/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.0644 - loss: 5.2139
Epoch 4/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.0741 - loss: 4.9979
Epoch 5/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.0921 - loss: 4.7723
Epoch 6/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.1088 - loss: 4.5906
Epoch 7/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 22s 30ms/step - accuracy: 0.1357 - loss: 4.3931
Epoch 8/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.1563 - loss: 4.1812
Epoch 9/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.1799 - loss: 4.0076
Epoch 10/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.2075 - loss: 3.8307
Epoch 11/150
692/692 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.2218 - loss: 3.7373
Epoch 12/150
692/69

### Text Generation

In [75]:
text_lenght= 15 # 15 words per line

def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index

            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break

            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)

        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)

    return general_text

In [76]:
input_text="Away"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

["is crazy love me say where you take to get there's all i'd see under",
 "me right journey it's again keep the change so yeah it be more when when",
 "you sometimes say i don't know they can didn't never take to day now of",
 "all the girls i've ever known it's you it's you you're my favorite is my",
 "mama don't like you and she likes when your side up though turning your latin",
 'for me the finer the can that the favorite my favorite my favorite my wind']

In [77]:
input_text="When your not with me"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

["any holding is you's your pain is breaking and all you're at standing nicki in",
 "every way i look then i are on it boy cause i'm going so was",
 "i a fool to let you break down my walls 'cause if you like the",
 'only thing that i got right now one day when the sky is my heart',
 "your whole beats they ah stop get whoa baby i'm no keep you or so",
 "i'm staying with you till you want me to go girl you don't understand what"]